# シンプルなファインチューニング例
- やりたいこと
  - [20 newsgroups text dataset](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset)を分類タスクとして学習したい。
- 方針
  - spacyにより学習済み英語モデル（en_core_web_sm）を用意する。
  - 学習済みモデルを用いて、20 newsgroups textの記事をベクトル化する。
  - 分類学習にはSVM、NNを用いる。

In [1]:
!date

Wed Jun 16 07:34:31 UTC 2021


## 事前学習

### 環境構築
- spacyと学習済みモデルをインストール。

In [2]:
!pip install -U ginza
!python -m spacy download en_core_web_sm
#!python -m spacy download en_core_web_lg

Requirement already up-to-date: ginza in /usr/local/lib/python3.7/dist-packages (4.0.6)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

### 事前学習により得られたモデルの確認
- 単語でも文章でもベクトル化できる。
- ベクトル化できたため、類似単語も確認可能。

In [4]:
# 動作確認1
nlp = spacy.load("en_core_web_sm")
token = nlp('artificial')
print(token.vector.shape)
print(token.vector[:5])
print(token.vector_norm)


(96,)
[-0.64146984 -1.346977   -1.4614831  -2.7170322   4.683545  ]
16.45753783733204


In [5]:
# 動作確認2
words = ['apple', 'banana', 'car']

tokens = []
for word in words:
  tokens.append(nlp(word))

for token1 in tokens:
  for token2 in tokens:
    if token1 == token2:
      continue
    else:
      sim = token1.similarity(token2)
      print("similarity({}, {}) = {}".format(token1.text, token2.text, sim))


similarity(apple, banana) = 0.7039913704898891
similarity(apple, car) = 0.5905734774861556
similarity(banana, apple) = 0.7039913704898891
similarity(banana, car) = 0.5619708709107428
similarity(car, apple) = 0.5905734774861556
similarity(car, banana) = 0.5619708709107428


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  del sys.path[0]


## ファインチューニング
事前学習済みモデルを用意できた。これを用いて本当にやりたい 20 news 分類学習に移る。

### データセットを用意
- 20 newsのデータセットを用意。

In [6]:
# fine-tuneing stage.
# デーセットの用意
# こちらも時間かかるので、変換したデータセットを指定した場所に保存。
# 既に保存済みデータセットの利用にも対応。

from sklearn.datasets import fetch_20newsgroups
#categories = ['alt.atheism', 'sci.space']
categories = ['comp.os.ms-windows.misc',  'comp.sys.mac.hardware',  'misc.forsale']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
train_text = newsgroups_train.data
train_label = newsgroups_train.target
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
test_text = newsgroups_test.data
test_label = newsgroups_test.target

### 事前学習モデルによるベクトル化
- en_core_web_sm によるベクトルを train_vectors, test_vectors に保存。

In [7]:
!date
# 事前学習済みモデルにより、文章をベクトルに変換
def sentence2vector(sentences, model):
    vectors = []
    for sent in sentences:
        vectors.append(nlp(sent).vector)
    return vectors

train_vectors = sentence2vector(train_text, nlp)
test_vectors = sentence2vector(test_text, nlp)

!date

Wed Jun 16 07:34:43 UTC 2021
Wed Jun 16 07:37:35 UTC 2021


### 分類学習

In [8]:
!date

#from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.neural_network import MLPClassifier

#clf1 = MultinomialNB()
clf2 = svm.SVC(gamma='scale')
clf3 = MLPClassifier(max_iter=1000)
clfs = {"SVM":clf2, "NN":clf3}

scores = []
for name, clf in clfs.items():
  clf.fit(train_vectors, train_label)
  score = clf.score(test_vectors, test_label)
  scores.append(score)
  print("score = {} by {}".format(score,name))

!date

Wed Jun 16 07:37:35 UTC 2021
score = 0.6706586826347305 by SVM
score = 0.6595380667236954 by NN
Wed Jun 16 07:37:49 UTC 2021


In [9]:
!date

Wed Jun 16 07:37:49 UTC 2021
